# MUSA 5090 Final Project
This file is used for data exploration, analysis, feature engineering, and model buildings.


## Datasets

In [7]:
from google.colab import drive
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# read in properties dataset
opa_dir = '/content/drive/MyDrive/musa-final-proj/opa_properties_public.csv'
properties = pd.read_csv(opa_dir)
properties.head(200)

<ipython-input-4-0149586cf5df>:3: DtypeWarning: Columns (4,16,25,27,48,76) have mixed types. Specify dtype option on import or set low_memory=False.
  properties = pd.read_csv(opa_dir)


,objectid,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,...,utility,view_type,year_built,year_built_estimate,zip_code,zoning,pin,building_code_new,building_code_description_new,shape
0,445763190,2022-05-24 12:13:13-04:00,NaN,NEC DOVER ST,NaN,SC,VACANT LAND COMMER < ACRE,6,VACANT LAND,172.0,...,NaN,I,NaN,NaN,19132.0,CMX1,1001321879,NaN,NaN,SRID=2272;POINT ( 2689039.51193106 252319.375...
1,445763191,2022-05-24 12:13:13-04:00,NaN,SEC 27TH ST,NaN,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,32.0,...,NaN,I,NaN,NaN,19146.0,RSA5,1001179696,NaN,NaN,SRID=2272;POINT ( 2686489.39071898 229330.622...
2,445763192,2022-05-24 12:13:13-04:00,NaN,NWC 16TH ST,NaN,T38,ROW B/OFF-STR 2 STY STONE,3,MIXED USE,373.0,...,NaN,I,1950.0,NaN,19145.0,CMX1,1001408261,820,ROW MIXED-COM/RES-BLT AS RES,SRID=2272;POINT ( 2690626.09996523 221499.086...
3,445763193,2022-05-24 12:13:13-04:00,NaN,"343'7 1/8"" W 31ST ST",NaN,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,169.0,...,NaN,I,NaN,NaN,19132.0,RSA5,1001168670,NaN,NaN,SRID=2272;POINT ( 2687281.27004074 250386.258...
4,445763194,2022-05-24 12:13:13-04:00,NaN,197' W OF 19TH ST,NaN,ZL0,MISC FUNERAL HOME MASONRY,4,COMMERCIAL,134.0,...,NaN,A,1920.0,NaN,19130.0,RM1,1001494378,246,FUNERAL HOME,SRID=2272;POINT ( 2691553.59778215 240243.694...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,445763289,2022-05-24 12:13:13-04:00,NaN,"334'11 1/4 "" N",NaN,SC,VACANT LAND COMMER < ACRE,6,VACANT LAND,164.0,...,NaN,I,NaN,NaN,19133.0,CMX2,1001598005,NaN,NaN,SRID=2272;POINT ( 2698753.98725273 249011.820...
196,445763290,2022-05-24 12:13:13-04:00,NaN,290' W OF 31ST ST,NaN,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,151.0,...,NaN,I,NaN,NaN,19121.0,RSA5,1001408622,NaN,NaN,SRID=2272;POINT ( 2687105.63915840 248735.226...
197,445763291,2022-05-24 12:13:13-04:00,NaN,"74'7 3/4""N TIOGA",NaN,SC,VACANT LAND COMMER < ACRE,6,VACANT LAND,201.0,...,NaN,I,NaN,NaN,19140.0,CMX2,1001625620,NaN,NaN,SRID=2272;POINT ( 2694809.80616473 256111.430...
198,445763292,2022-05-24 12:13:13-04:00,NaN,SW COR 50TH ST,NaN,S30,ROW W/OFF STR 2 STY MASON,3,MIXED USE,104.0,...,NaN,I,1925.0,NaN,19139.0,RSA5,1001412740,820,ROW MIXED-COM/RES-BLT AS RES,SRID=2272;POINT ( 2677389.92105706 240959.320...


Upon checking the dataset, we found the following attributes from the opa properties  might be useful to predicting property values.
- basements
- category code
- census tract
- central air
- exterior condition
- fireplaces
- garage spacess
- garage type
- interior condition
- market value
- number of bedrooms
- number of bathrooms
- number of rooms
- number stories
- parcel number
- quality grade
- sale price
- total area
- total livable area
- view
- year built
- shape (geometry)


In [14]:
# some baseline data processing work
properties_mdl = properties[['basements', #
                          'category_code',
                          'census_tract',
                          'central_air', #
                          'fireplaces', #
                          'garage_type', #
                          'market_value',
                          'number_of_bedrooms', #
                          'number_of_bathrooms', #
                          'number_of_rooms', #
                          'number_stories',#
                          'parcel_number',
                          'quality_grade', #
                          'sale_price',
                          'type_heater', #
                          'total_area', #
                          'total_livable_area', #
                          'view_type', #
                          'building_code_description_new', #
                          'year_built']] #

In [17]:
properties_mdl['Age'] = 2024 - properties_mdl['year_built']

properties_mdl['numRooms'] = np.select(
    [(properties_mdl['number_of_bedrooms'].isna()) & (~properties_mdl['number_of_bathrooms'].isna()),
     (properties_mdl['number_of_bathrooms'].isna()) & (~properties_mdl['number_of_bedrooms'].isna()),
      (properties_mdl['number_of_bathrooms'].isna()) & (properties_mdl['number_of_bedrooms'].isna())],
    [properties_mdl['number_of_bathrooms'],
     properties_mdl['number_of_bedrooms'],
     0],
    default=properties_mdl['number_of_bedrooms'] + properties_mdl['number_of_bathrooms']
)

properties_mdl['hasAC'] = np.where(properties_mdl['central_air'].isin(['1', 'Y']), 'Y', 'N')
properties_mdl['hasBasement'] = np.where(properties_mdl['basements'].isin(['1', '4', 'A', 'B', 'C', 'D', 'E', 'F']), 'Y', 'N')
properties_mdl['hasFireplace'] = np.where((properties_mdl['fireplaces'] == 0) | (properties_mdl['fireplaces'].isna()), 'N', 'Y')
properties_mdl['hasGarage'] = np.where((properties_mdl['garage_type'] == 0) | (properties_mdl['garage_type'].isna()), 'N', 'Y')
properties_mdl['stories'] = np.where(properties_mdl['number_stories'] == 1, 'single', np.where(properties_mdl['number_stories'] == 2, 'double', 'multiple'))
properties_mdl['area'] = np.where(properties_mdl['total_livable_area'] > properties_mdl['total_area'], properties_mdl['total_livable_area'], properties_mdl['total_area'])
properties_mdl['hasHeater'] = np.where((properties_mdl['type_heater'] == 0) | (properties_mdl['type_heater'].isna()), 'N', 'Y')
properties_mdl['quality'] = np.where(properties_mdl['quality_grade'].isin(['4', '5', '6', 'A', 'A+', 'A-', 'B', 'B+', 'B-', 'S', 'S+', 'X-']), 'Good', 'Bad')


properties_mdl['view'] = np.select(
    [properties_mdl['view_type'].isin(['I', '0']) | properties_mdl['view_type'].isna(),
     properties_mdl['view_type'].isin(['A', 'B', 'C'])],
    ['Typical','Scenic'],
    default='Urban'
)

properties_mdl['logarea'] = np.log(properties_mdl['area'])

<ipython-input-17-c59550975f01>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties_mdl['Age'] = 2024 - properties_mdl['year_built']
<ipython-input-17-c59550975f01>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties_mdl['numRooms'] = np.select(
<ipython-input-17-c59550975f01>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [21]:
# Convert the pandas Series of boolean values to numpy ndarray using .values
condition1 = properties_mdl['building_code_description_new'].str.contains('ROW', case=False).fillna(False).values
condition2 = properties_mdl['building_code_description_new'].str.contains('TWIN', case=False).fillna(False).values

properties_mdl['buildingdis'] = np.select(
    [
        condition1,
        condition2
    ],
    [
        'Row',
        'TWIN'
    ],
    default='Other'
)


<ipython-input-21-81fd3ebc4411>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties_mdl['buildingdis'] = np.select(


In [40]:
properties_mdl['sale_price'].describe()

count    5.272400e+05
mean     1.273221e+05
std      1.995874e+05
min      0.000000e+00
25%      1.000000e+00
50%      5.886000e+04
75%      1.749000e+05
max      1.999990e+06
Name: sale_price, dtype: float64

In [41]:
properties_mdl = properties_mdl[
    (properties_mdl['Age'] < 500) &
    (properties_mdl['sale_price'] < 2000000) &
    (properties_mdl['sale_price'] > 10000) &  # Include the condition for sale_price
    (properties_mdl['numRooms'] < 30) &
    (properties_mdl['total_livable_area'] != 0) &
    (~properties_mdl['total_area'].isna()) &
    (properties_mdl['area'] < 50000)]


In [42]:
properties.shape[0]

582957

In [43]:
properties_mdl.shape[0]

356342

In [44]:
import statsmodels.api as sm

X = properties_mdl[['Age', 'numRooms', 'hasBasement', 'hasAC', 'quality', 'buildingdis', 'hasFireplace', 'hasGarage', 'stories', 'logarea', 'view']]
y = properties_mdl['sale_price']
# Display the data types of each column in X
print(X.dtypes)

# Convert non-numeric columns to numeric, if possible
X_numeric = X.apply(pd.to_numeric, errors='ignore')

# Check if there are any non-numeric columns left
non_numeric_columns = X_numeric.select_dtypes(exclude=['number']).columns
if len(non_numeric_columns) > 0:
    print("Non-numeric columns:", non_numeric_columns)

# If there are still non-numeric columns, you need to handle them appropriately
# For example, you can drop them or encode them using one-hot encoding

# Add a constant term to the independent variables
X_numeric = sm.add_constant(X_numeric)
X_encoded = pd.get_dummies(X_numeric, drop_first=True)

# Fit the linear regression model
reg1 = sm.OLS(y, X_encoded).fit()

# Get the summary statistics
summary_reg1 = reg1.summary()

print(summary_reg1)


Age             float64
numRooms        float64
hasBasement      object
hasAC            object
quality          object
buildingdis      object
hasFireplace     object
hasGarage        object
stories          object
logarea         float64
view             object
dtype: object
Non-numeric columns: Index(['hasBasement', 'hasAC', 'quality', 'buildingdis', 'hasFireplace',
       'hasGarage', 'stories', 'view'],
      dtype='object')
                            OLS Regression Results                            
Dep. Variable:             sale_price   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.262
Method:                 Least Squares   F-statistic:                     9043.
Date:                Wed, 03 Apr 2024   Prob (F-statistic):               0.00
Time:                        15:30:49   Log-Likelihood:            -4.8319e+06
No. Observations:              356342   AIC:                         9.664e+06
Df Residuals:

In [45]:
# try random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize the Random Forest model
rf_model = RandomForestRegressor()

# Fit the model to the training data
rf_model.fit(X_train, y_train)

# Make predictions on the testing data
rf_predictions = rf_model.predict(X_test)

# Calculate Mean Squared Error (MSE)
rf_mse = mean_squared_error(y_test, rf_predictions)
print("Random Forest Mean Squared Error:", rf_mse)


Random Forest Mean Squared Error: 27389090411.52483


In [46]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the MLP model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer with one neuron for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on the testing data
y_pred = model.predict(X_test_scaled)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)


Epoch 1/50
7127/7127 [==============================] - 25s 3ms/step - loss: 47436472320.0000 - val_loss: 34351159296.0000
Epoch 2/50
7127/7127 [==============================] - 19s 3ms/step - loss: 34152699904.0000 - val_loss: 33786771456.0000
Epoch 3/50
7127/7127 [==============================] - 23s 3ms/step - loss: 33729110016.0000 - val_loss: 33458165760.0000
Epoch 4/50
7127/7127 [==============================] - 19s 3ms/step - loss: 33413384192.0000 - val_loss: 33149462528.0000
Epoch 5/50
7127/7127 [==============================] - 22s 3ms/step - loss: 33111486464.0000 - val_loss: 32838850560.0000
Epoch 6/50
7127/7127 [==============================] - 21s 3ms/step - loss: 32801413120.0000 - val_loss: 32503420928.0000
Epoch 7/50
7127/7127 [==============================] - 22s 3ms/step - loss: 32468121600.0000 - val_loss: 32152971264.0000
Epoch 8/50
7127/7127 [==============================] - 21s 3ms/step - loss: 32095676416.0000 - val_loss: 31754862592.0000
Epoch 9/50
7127/

In [ ]:
# check attribtue name
properties.columns.tolist()

['objectid',
 'assessment_date',
 'basements',
 'beginning_point',
 'book_and_page',
 'building_code',
 'building_code_description',
 'category_code',
 'category_code_description',
 'census_tract',
 'central_air',
 'cross_reference',
 'date_exterior_condition',
 'depth',
 'exempt_building',
 'exempt_land',
 'exterior_condition',
 'fireplaces',
 'frontage',
 'fuel',
 'garage_spaces',
 'garage_type',
 'general_construction',
 'geographic_ward',
 'homestead_exemption',
 'house_extension',
 'house_number',
 'interior_condition',
 'location',
 'mailing_address_1',
 'mailing_address_2',
 'mailing_care_of',
 'mailing_city_state',
 'mailing_street',
 'mailing_zip',
 'market_value',
 'market_value_date',
 'number_of_bathrooms',
 'number_of_bedrooms',
 'number_of_rooms',
 'number_stories',
 'off_street_open',
 'other_building',
 'owner_1',
 'owner_2',
 'parcel_number',
 'parcel_shape',
 'quality_grade',
 'recording_date',
 'registry_number',
 'sale_date',
 'sale_price',
 'separate_utilities',
 '

In [47]:
y_test.describe()

count    7.126900e+04
mean     1.888969e+05
std      2.182241e+05
min      1.000600e+04
25%      5.600000e+04
50%      1.250000e+05
75%      2.400000e+05
max      1.992500e+06
Name: sale_price, dtype: float64

In [ ]:
# Read in census data
import requests

# Your API key
api_key = "77ba26a94bf747d61761544063606374da01af62"

# Base URL for Census API
url = "https://api.census.gov/data/2019/acs/acs5"

# Parameters for the API request
# Adjusting to fetch data for all census tracts within Philadelphia County (county code 101) in Pennsylvania (state code 42)
params = {
    "get": "B01003_001E",  # Total population
    'for': 'tract:*',  # All tracts
    'in': 'state:42 county:101',  # Within Philadelphia County, Pennsylvania
    "key": api_key
}

# Make the API request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to JSON
    data = response.json()
    # Create a pandas DataFrame
    # The DataFrame includes the total population for each tract in Philadelphia County
    census = pd.DataFrame(data[1:], columns=data[0])
    # Optionally, convert tract and county codes to numeric and sort
    census[['state', 'county', 'tract']] = df[['state', 'county', 'tract']].apply(pd.to_numeric)
    census.sort_values(by=['tract'], inplace=True)
    print("Successful Download")
else:
    print(f"Failed to fetch data: {response.status_code}")

Successful Download
